## Tuning Parquet files

For high performance, Parquet requires a good schema and physical design. The Parquet libraries try to infer a working schema and design, but it is sometimes not efficient and sometimes also not working on the data. Let's experiment with Parquet

* Data types
* Encodings
* And compressions

and then check how an optimized design performs in querying.

### Parquet data types

**Physical data types** are the binary data types that Parquet stores data in:

- `BOOLEAN`: 1 bit boolean
- `INT32`: 32 bit signed ints
- `INT64`: 64 bit signed ints
- `FLOAT`: IEEE 32-bit floating point values
- `DOUBLE`: IEEE 64-bit floating point values
- `BYTE_ARRAY`: arbitrarily long byte arrays
- `FIXED_LEN_BYTE_ARRAY`: fixed length byte arrays

**Logical data types** are additional interpretations on top of the physical data types for outside representation and validation:

Numeric Types:

- `INT(bitWidth, isSigned)`: Signed/unsigned integers (8, 16, 32, 64 bits)
  - `INT8`, `INT16`, `INT32`, `INT64` (signed)
  - `UINT8`, `UINT16`, `UINT32`, `UINT64` (unsigned)
- `DECIMAL(precision, scale)`: Fixed-point decimal numbers (mapped to INT32, INT64, FIXED_LEN_BYTE_ARRAY)

String Types:

- `STRING`: UTF-8 encoded character strings
- `ENUM`: Enumerated string values
- `UUID`: 128-bit universally unique identifiers
- `JSON`: JSON-encoded strings

Temporal Types:

- `DATE`: Calendar date (days since Unix epoch)
- `TIME`: Time of day with microsecond or nanosecond precision
  - With/without timezone
- `TIMESTAMP`: Instant in time with microsecond or nanosecond precision
  - With/without timezone (UTC or local)
- `INTERVAL`: Time duration

Binary Types:

- `BSON`: Binary JSON format

Nested/Complex Types:

- `LIST`: Ordered collection of elements
- `MAP`: Key-value pairs
- `STRUCT`: Record with named fields (nested columns)

We'll review the nested types in the next notebook. Let's first try to give our event data better data types and a better schema first.

Note: The notebook expects the files generated by the first notebook to be present for comparison purposes.

In [ ]:
import re
import dateutil
from time import time
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import duckdb
from pathlib import Path
from IPython.display import display, HTML
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 160)

%reload_ext autoreload
%autoreload 2
from helpers import read_jsonl, inspect, compare_sizes

events_data_path = Path('../data/input/events.jsonl')
events_data = read_jsonl(events_data_path)
events_parquet_path = Path('../data/output/events.parquet')
events_parquet = pq.ParquetFile(events_parquet_path)

In [ ]:
events_data_typed = []
for record in events_data:
    events_data_typed.append({
        'creationTime': dateutil.parser.isoparse(record['creationTime']),
        'id': int(record['id']),
        'source': int(record['source']),
        'text': record['text'],
        'time': dateutil.parser.isoparse(record['time']),
        'type': record['type'],
    })

events_table_typed = pa.Table.from_pylist(
    events_data_typed,
    schema=pa.schema([
        ('creationTime', pa.timestamp("ms")),
        ('id', pa.int64()),
        ('source', pa.int64()),
        ('text', pa.string()),
        ('time', pa.timestamp("ms")),
        ('type', pa.string())
    ])
)

events_parquet_path_typed = Path('../data/output/events_typed.parquet')
pq.write_table(events_table_typed, events_parquet_path_typed)
inspect(events_parquet_path_typed)
events_parquet_typed = pq.ParquetFile(events_parquet_path_typed)

compare_sizes(events_parquet, "Original", events_parquet_typed, "Typed")
print(f"Original size: {events_parquet_path.stat().st_size} "
      f"Limited size: {events_parquet_path_typed.stat().st_size} "
      f"Reduction to: {events_parquet_path_typed.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}%")

The dictionaries grew signifcantly. What could be the reason? Hint: How well do you expect text and binary timestamps to compress with, for example, "gzip"?

Note: If you inspect the generated files with `parquet-tools`, it reports that while a dictionary is created, only the first data page actually uses the dictionary and all other data pages do not. The Java Parquet implementation is a little smarter and immediately skips dictionaries if they are not proving efficient.

TBD: The timestamp seems to be now interpreted in the local timezone. The timestamps are output one hour later than with the untyped string.

### Controlling Dictionary Encoding

Let's use dictionaries only where they make sense, i.e., only where properties have similar values most of the time. What columns would that be?

In [ ]:
# Define the schema with explicit dictionaries
events_table_dict = pa.Table.from_pylist(
    events_data_typed,
    schema=pa.schema([
        ('creationTime', pa.timestamp("ms")),
        ('id', pa.int64()),
        ('source', pa.dictionary(pa.int8(), pa.int64())),
        ('text', pa.string()),
        ('time', pa.timestamp("ms")),
        ('type', pa.dictionary(pa.int8(), pa.string())),
    ])
)

# Write using only the "source" and "type" column as dictionary-encoded
events_parquet_path_dict = Path('../data/output/events_dict.parquet')
pq.write_table(events_table_dict, events_parquet_path_dict, use_dictionary=['source', 'type'])

events_parquet_dict = pq.ParquetFile(events_parquet_path_dict)
compare_sizes(events_parquet, "Original", events_parquet_dict, "Limited")
print(f"Original size: {events_parquet_path.stat().st_size} "
      f"Limited size: {events_parquet_path_dict.stat().st_size} "
      f"Reduction to: {events_parquet_path_dict.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}%")

Observations:

* Normal compression does a good job most of the time.
* By setting a schema and controlling dictionary usage, we could reduce the file size to 80%.

Note: You can view more details on dictionaries and data pages using `parquet-tools`. For example, to see `source` (Column 2), and to view its dictionary (usually in Page 0):

```
parquet-tools inspect --row-group 0 --column-chunk 2 events.parquet | python3 -mjson.tool 
parquet-tools inspect --row-group 0 --column-chunk 2 --page 0 events.parquet | python3 -mjson.tool 
```




### Sorting order

Assume that our predominant use case is to query the data by `source` and `time`. If we sort by these properties, the query engine can narrow down the data pages to be read. 

Also, Parquet supports run-length encoding and delta encoding. 

* **Run-length encoding** means that repeating values are stored as a tuple of (number of repetitions, value). Conceptually, if your values are "device_1", "device_1", "device_2", "device_2", the run-length encoding is (2, "device_1"), (2, "device_2").
* **Delta encoding** means that the difference between two values is stored. For example, if you have data coming into the system every second for a device, instead of storing each timestamp, only the initial timestamp and a sequence of 1000 (milliseconds) is stored. This way, you get many similar values that can be efficiently run-length encoded.

Note: Delta-encoding is not efficient for timestamps on events usually, as they are not regularly emitted. We'll get back to that later. It still helps a bit here.

In [ ]:
events_data_sorted = sorted(events_data_typed, key=lambda x: (x['source'], x['time']))
events_table_sorted = pa.Table.from_pylist(
  events_data_sorted,
  schema=pa.schema([
    ('creationTime', pa.timestamp("ms")),
    ('id', pa.int64()),
    ('source', pa.dictionary(pa.int8(), pa.int64())),
    ('text', pa.string()),
    ('time', pa.timestamp("ms")),
    ('type', pa.dictionary(pa.int8(), pa.string())),
  ])
)

events_parquet_path_sorted = Path('../data/output/events_sorted.parquet')
pq.write_table(events_table_sorted, events_parquet_path_sorted, column_encoding={'time':'DELTA_BINARY_PACKED'}, use_dictionary=['source', 'type'])

events_parquet_dict = pq.ParquetFile(events_parquet_path_sorted)
compare_sizes(events_parquet, "Original", events_parquet_dict, "Limited")
print(f"Original size: {events_parquet_path.stat().st_size} "
      f"Limited size: {events_parquet_path_sorted.stat().st_size} "
      f"Reduction to: {events_parquet_path_sorted.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}%")

What can you observe on the "source" and "time" column? Verify using `parquet-tools`:

```
parquet-tools inspect --row-group 0 --column-chunk 2 events_sorted.parquet | python3 -mjson.tool 
parquet-tools inspect --row-group 0 --column-chunk 2 --page 0 events_sorted.parquet | python3 -mjson.tool 
```

### Data modelling

In our example, events contain formatted log messages with a text and the ID of the work piece. 

```
Starting to work on workpiece 2024_9550021
Stop to work on workpiece 2024_9550021
```

This is a common approach, but for analytics, it would be much more efficient to separate the workpiece ID right away instead of first formatting it and then parsing it back. Let's split the log entry and add a column for the workpiece ID.

In [ ]:

workpiece_pattern = re.compile(r'(.*?workpiece\s+)([\w_]+)')

events_data_split = []
for record in events_data_sorted:
    new_record = record.copy()
    text = new_record['text']
    match = workpiece_pattern.match(text)

    if match:
        new_record['text'] = match.group(1).strip()
        new_record['workpiece_id'] = match.group(2)
    else:
        new_record['workpiece_id'] = None

    events_data_split.append(new_record)

table_split = pa.Table.from_pylist(
    events_data_split,
    schema=pa.schema([
    ('creationTime', pa.timestamp("ms")),
    ('id', pa.int64()),
    ('source', pa.dictionary(pa.int8(), pa.int64())),
    ('text', pa.dictionary(pa.int8(), pa.string())), # Text is now low-cardinality
    ('time', pa.timestamp("ms")),
    ('type', pa.dictionary(pa.int8(), pa.string())),
    ('workpiece_id', pa.string()) # workpiece_id is high-cardinality
]))

events_parquet_path_split = Path('../data/output/events_split.parquet')
start_time = time()
pq.write_table(
    table_split,
    events_parquet_path_split,
    column_encoding={'time': 'DELTA_BINARY_PACKED'},
    use_dictionary=['source', 'type', 'text'] # Add the new 'text' to dictionary encoding
)
end_time = time()
snappy_duration = end_time - start_time

events_parquet_split = pq.ParquetFile(events_parquet_path_split)
print("Comparing original file with the new split-text file:")
compare_sizes(events_parquet, "Original", events_parquet_split, "Split")
print(f"\nOriginal size: {events_parquet_path.stat().st_size} bytes")
print(f"Split file size: {events_parquet_path_split.stat().st_size} bytes")
print(f"Reduction to: {events_parquet_path_split.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}% of original")

### Compression

gzip can be used instead of snappy as compression.

In [ ]:
events_parquet_path_gzipped = Path('../data/output/events_gzipped.parquet')
start_time = time()
pq.write_table(table_split, events_parquet_path_gzipped, compression='gzip', column_encoding={'time':'DELTA_BINARY_PACKED'}, use_dictionary=['source', 'type', 'text'])
end_time = time()
gzip_duration = end_time - start_time

events_parquet_dict = pq.ParquetFile(events_parquet_path_gzipped)
compare_sizes(events_parquet, "Original", events_parquet_dict, "Limited")
print(f"Original size: {events_parquet_path.stat().st_size}")
print(f"Snappy size: {events_parquet_path_split.stat().st_size} {events_parquet_path_split.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}%")
print(f"Gzipped size: {events_parquet_path_gzipped.stat().st_size} {events_parquet_path_gzipped.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}% ")
print(f"Snappy write time: {snappy_duration:.2f} seconds")
print(f"Gzipped write time: {gzip_duration:.2f} seconds")

Much better compression, but also very heavy performance impact -- also for querying!

Note:
* If you use gzip on original JSON file, you will reach 23475269 or around double the size of the optimized file.
* If you use gzip encoding on the first Parquet file, you will reach around 60% compression.

## Querying

Let's query the brandnew workpiece ID. 

In [ ]:
con = duckdb.connect()

query = f"""
SELECT
    DATE_TRUNC('week', time) as week,
    COUNT(distinct workpiece_id) as pieces_per_week
FROM '{events_parquet_path_split}'
GROUP BY week
ORDER BY week
"""
result = con.execute(query).fetchdf()

plt.figure(figsize=(12, 6))
plt.plot(result['week'], result['pieces_per_week'], marker='o', linewidth=2, markersize=6)
plt.xlabel('Week', fontsize=12)
plt.ylabel('Workpieces per Week', fontsize=12)
plt.title('Weekly Workpiece Production', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

con.close()


## Summary

PyArrow's schema discovery does a "quick and dirty" job in discovering the schema in the source data and blindly applies dictionaries everywhere, but does not even consistently use them. The Java implementation is a little bit smarter, but with the very basic type system of JSON as input, there is only so much that you can do automatically. So for analytics on larger scale, some tuning is needed -- we'll get back to that in the third module.

Other insights: 
* Parquet uses only a few physical types in the column storage layer; all other standard types are applied on top.
* "Snappy" compression seems a good tradeoff between disk and CPU resource usage.
